## 7. This Jupyter Notebook shows the seventh iteration. All previous iterations have used the Washington Monthly dataset, with the Washington Monthly ranks as targets and the Washington Monthly features as features. This iteration tests the predictiveness of the Washington Monthly features on the US News rankings. Therefore the US News rankings are added in as a target for this iteration. 

In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import r2_score


In [2]:
# Loading data
file_path = Path("Resources/2022-College-Guide-Main-Rankings-clear-plus-USNews.csv")
df_rankings = pd.read_csv(file_path)
df_rankings.head()


,USNews rank,UnitID,WMRank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,...,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank,Unnamed: 41
0,3.0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,...,0.290903,10,NO,3,0.068361,401,3,2,187,NaN
1,7.0,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,...,0.160573,71,YES,6,0.246743,209,2,9,40,NaN
2,2.0,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,...,0.065477,332,NO,2,0.000000,436,5,1,324,NaN
3,1.0,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,...,0.064500,339,NO,6,0.000000,436,7,6,224,NaN
4,10.0,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,...,0.089523,224,YES,5,0.170164,305,12,15,20,NaN


In [3]:
df_rankings.columns

Index(['USNews rank', 'UnitID', 'WMRank', 'Name', '8-year graduation rate',
       'Graduation rate rank',
       'Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.',
       'Graduation rate performance rank', 'Pell/non-Pell graduation gap',
       'Pell graduation gap rank', 'Number of Pell graduates',
       'Actual vs. predicted Pell enrollment', 'Pell performance rank',
       'Median earnings 10 years after entering college',
       'Predicted median earnings 10 years after entering college',
       'Earnings performance rank',
       'Net price of attendance for families below $75,000 income',
       'Net price rank', '% of loan principal remaining 5 years later',
       'Repayment rank', 'Predicted principal remaining',
       'Repayment rate performance rank', 'Research expenditures, in millions',
       'Research expenditures rank', 'Bachelor's to PhD rank',
       'Science & engineering PhDs awarded', 'Science & engineering PhDs rank',
       'Facul

In [4]:
df_rankings.dtypes

USNews rank                                                                     float64
UnitID                                                                            int64
WMRank                                                                            int64
Name                                                                             object
8-year graduation rate                                                          float64
Graduation rate rank                                                              int64
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    float64
Graduation rate performance rank                                                  int64
Pell/non-Pell graduation gap                                                    float64
Pell graduation gap rank                                                          int64
Number of Pell graduates                                                        float64
Actual vs. predicted Pell enroll

In [5]:
df_rankings.count()

USNews rank                                                                     166
UnitID                                                                          442
WMRank                                                                          442
Name                                                                            442
8-year graduation rate                                                          442
Graduation rate rank                                                            442
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    442
Graduation rate performance rank                                                442
Pell/non-Pell graduation gap                                                    442
Pell graduation gap rank                                                        442
Number of Pell graduates                                                        442
Actual vs. predicted Pell enrollment                                        

In [6]:
# Drop last column
df_rankings.drop('Unnamed: 41', axis=1, inplace=True)

In [7]:
# Encoding Carnegie column (0=NO, 1=YES)

le = LabelEncoder()
df2_rankings = df_rankings.copy()
df2_rankings['Earns Carnegie Community Engagement Classification?'] = le.fit_transform(df2_rankings['Earns Carnegie Community Engagement Classification?'])


In [8]:
df2_rankings.isna().sum()

USNews rank                                                                     276
UnitID                                                                            0
WMRank                                                                            0
Name                                                                              0
8-year graduation rate                                                            0
Graduation rate rank                                                              0
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.      0
Graduation rate performance rank                                                  0
Pell/non-Pell graduation gap                                                      0
Pell graduation gap rank                                                          0
Number of Pell graduates                                                          0
Actual vs. predicted Pell enrollment                                        

In [9]:
# Dropping rows with NaNs in USNews rank column

df2_rankings.dropna(inplace=True)

In [10]:
df2_rankings.count()

USNews rank                                                                     166
UnitID                                                                          166
WMRank                                                                          166
Name                                                                            166
8-year graduation rate                                                          166
Graduation rate rank                                                            166
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    166
Graduation rate performance rank                                                166
Pell/non-Pell graduation gap                                                    166
Pell graduation gap rank                                                        166
Number of Pell graduates                                                        166
Actual vs. predicted Pell enrollment                                        

In [11]:
# Define the features set.
X = df2_rankings.copy()

# Dropping target columns and unique identifier columns, etc
X = X.drop(columns=["UnitID", "WMRank", "Name", "USNews rank"])
X.head()


,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,Actual vs. predicted Pell enrollment,Pell performance rank,Median earnings 10 years after entering college,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,0.963302,6,0.949654,187,-0.021703,46,260.6667,0.038096,122,97798,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,0.964294,5,0.938620,149,-0.041917,104,376.6667,-0.000590,211,103246,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,0.950971,10,0.990213,334,-0.032985,72,182.6667,0.076769,65,111222,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,0.976861,2,0.947812,137,-0.052740,132,153.3333,0.062997,85,95689,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,0.954474,9,0.967952,267,-0.014278,36,231.6667,0.028513,142,93115,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [12]:
# Checking datatypes to make sure there are no strings
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 0 to 434
Data columns (total 37 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   8-year graduation rate                                                        166 non-null    float64
 1   Graduation rate rank                                                          166 non-null    int64  
 2   Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.  166 non-null    float64
 3   Graduation rate performance rank                                              166 non-null    int64  
 4   Pell/non-Pell graduation gap                                                  166 non-null    float64
 5   Pell graduation gap rank                                                      166 non-null    int64  
 6   Number of Pell graduates          

In [13]:
# Define the target set.
y = df2_rankings["USNews rank"].ravel()
y[:5]


array([ 3.,  7.,  2.,  1., 10.])

In [14]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [15]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (124, 37)
Training Labels Shape: (124,)
Testing Features Shape: (42, 37)
Testing Labels Shape: (42,)


## Random Forest Regressor

In [16]:
# Instantiate model with 1000 decision trees
rf_model = RandomForestRegressor(n_estimators = 500, random_state = 42)
# Train the model on training data
rf_model=rf_model.fit(X_train_scaled, y_train)

In [17]:
# Making predictions using the testing data.
rf_predictions = rf_model.predict(X_test_scaled)


In [18]:
# Evaluate the performance with r2 score
r2_rf= r2_score(y_test,rf_predictions)
print('The r2 score for this Random Forest Regressor model is:', r2_rf)
#r2 score with 1000 estimators/ random state 42 = .964; 500 estimators .963; 1500 estimators .964

The r2 score for this Random Forest Regressor model is: 0.27998798103433986


In [19]:
# Getting accuracy data with Regressor/continuous variable model

# Calculate the absolute errors
errors = abs(rf_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Mean Absolute Error: 24.79
Accuracy: -inf %.


C:\Users\David\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


In [20]:
y_test

array([ 97.,  38., 121.,   0., 127.,  44., 137.,  49., 115.,  20.,  97.,
        32., 137., 127.,  41.,  83.,  17.,  51.,  72., 127.,  36.,  22.,
       115.,   0.,  72., 105.,  97.,   0., 137.,   0.,  44.,  44., 115.,
        38., 121., 127.,  67.,  83.,  62.,  83.,  77., 105.])

In [21]:
errors

array([  9.368,  10.244,  50.012,  83.744,  25.614,   6.83 ,  55.274,
         9.498,  39.364,   2.064,  22.58 ,  25.344,  23.98 ,  44.776,
         2.392,  13.222,   5.062,  15.27 ,   0.998,  63.05 ,   5.86 ,
        12.27 ,  14.06 ,  85.752,   9.138,  32.262,   9.076, 116.968,
        16.154,  80.266,   7.736,   3.15 ,  24.848,   9.924,   8.746,
        30.112,   9.55 ,   0.846,   1.426,   0.134,  14.744,  39.29 ])

In [22]:
from sklearn import metrics
# https://machinelearningmastery.com/regression-metrics-for-machine-learning

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, rf_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, rf_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, rf_predictions)))

Mean Absolute Error: 24.785666666666668
Mean Squared Error: 1338.294992857143
Root Mean Squared Error: 36.58271439979738


In [24]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


array([0.1417408 , 0.14764059, 0.12140434, 0.01832099, 0.00651342,
       0.00823587, 0.01214691, 0.02769649, 0.02791383, 0.02853409,
       0.03299306, 0.01663413, 0.01366477, 0.01250188, 0.00624855,
       0.0065813 , 0.06695952, 0.02013069, 0.00977675, 0.01059005,
       0.03305824, 0.00683164, 0.00692792, 0.04403507, 0.0102944 ,
       0.02183849, 0.006418  , 0.00855969, 0.00801762, 0.00707042,
       0.00115447, 0.0120524 , 0.02825404, 0.03120088, 0.00790247,
       0.01995712, 0.01019908])

In [26]:
# Sort the features by their importance.
sorted_list=sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
sorted_df=pd.DataFrame(sorted_list, columns=['Importance', 'Feature'])
sorted_df

,Importance,Feature
0,0.147641,Graduation rate rank
1,0.141741,8-year graduation rate
2,0.121404,Predicted graduation rate based on % of Pell r...
3,0.066960,Predicted principal remaining
4,0.044035,Faculty receiving significant awards
5,0.033058,Bachelor's to PhD rank
6,0.032993,Predicted median earnings 10 years after enter...
7,0.031201,Service-oriented majors rank
8,0.028534,Median earnings 10 years after entering college
9,0.028254,% of grads with service-oriented majors


In [27]:
#sorted_df.to_csv('Feature_Importances_USNews.csv')  

## Support Vector Regression

In [28]:
# Create a SVM model

# Instantiate a SVM model
from sklearn.svm import SVR

# Fit the data
regressor = SVR(kernel = 'poly')
svm_model = regressor.fit(X_train, y_train)
svm_model

#Testing different kernels for improved r2 score
#rbf kernel r2 score = .103
#linear kernel r2 score = -209.628
#sigmoid kernel r2 = -.013
#poly(polynomial) kernel r2= .313


SVR(kernel='poly')

In [29]:
# Predicting the Test set results
svm_predictions = svm_model.predict(X_test)

In [30]:
from sklearn.metrics import r2_score
r2_svm=r2_score(y_test, svm_predictions)
print('The r2 score for this SVM model is:', r2_svm)

The r2 score for this SVM model is: 0.09232034638014097


## Decision Tree Regression

In [32]:
# Training the Decision Tree Regression model on the Training set
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state = 0)
dt_model = dt_model.fit(X_train, y_train)
dt_model

DecisionTreeRegressor(random_state=0)

In [33]:
# Predicting the Test set results
dt_predictions = dt_model.predict(X_test)

In [34]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_dt= r2_score(y_test,dt_predictions)
print('The r2 score for this Decision Tree model is:', r2_dt)

The r2 score for this Decision Tree model is: -0.7558753775808471


## Multiple Linear Regression

In [35]:
# Training the multiple regression model on the Training set
from sklearn.linear_model import LinearRegression
mlr_model = LinearRegression()
mlr_model = mlr_model.fit(X_train, y_train)
mlr_model

LinearRegression()

In [36]:
# Predicting the Test set results
mlr_predictions = mlr_model.predict(X_test)

In [37]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_mlr= r2_score(y_test,mlr_predictions)
print('The r2 score for this Multiple Linear Regression model is:', r2_mlr)

The r2 score for this Multiple Linear Regression model is: -0.15071218369092443
